In [2]:

import pickle as pkl
import numpy as np
import keras
import boto3
import numpy as np
import pandas as pd
import os
import json
from datetime import timedelta
from dateutil import parser
import boto3
import urllib.request
import datetime

Using TensorFlow backend.


In [0]:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1xhqQarXaEpWajnM2z48-9XUgnoIrxM5G"})   # replace the id with id of file you want to access
downloaded.GetContentFile('x_master.p') 


In [0]:
downloaded2 = drive.CreateFile({'id':"1pnyVbLd61mnMslPGYwOB-wGFmK_lJ-pb"})   # replace the id with id of file you want to access
downloaded2.GetContentFile('y_master.p') 

# https://drive.google.com/open?id=1pnyVbLd61mnMslPGYwOB-wGFmK_lJ-pb

In [0]:
downloaded3 = drive.CreateFile({'id':"1yAWKsQPmfZZ2ycmxG6NbtYkLQIqhl8MS"})   # replace the id with id of file you want to access
downloaded3.GetContentFile('x_test1911.p') 

#https://drive.google.com/open?id=1yAWKsQPmfZZ2ycmxG6NbtYkLQIqhl8MS

In [0]:
downloaded3 = drive.CreateFile({'id':"1odizDSpF_gVc25cPYSPu3KQfvSbLgH-G"})   # replace the id with id of file you want to access
downloaded3.GetContentFile('y_test1911.p') 

#https://drive.google.com/open?id=1odizDSpF_gVc25cPYSPu3KQfvSbLgH-G

In [0]:
X = pkl.load(open("x_master.p", "rb"))
Y = pkl.load(open("y_master.p", "rb"))
x_test1911 = pkl.load(open("x_test1911.p", "rb"))
y_test1911 = pkl.load(open("y_test1911.p", "rb"))

In [0]:
x_bin=[]
y_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(Y):
  if X[i].shape == (160, 4):
      if y in [1,2,3]:
        y_bin.append(1)
        x_bin.append(X[i])
        smoking_event+=1
      elif y >= 20:
        y_bin.append(0)
        x_bin.append(X[i])
        non_smoking_event+=1
        
x_bin=np.array(x_bin)
y_bin=np.array(y_bin)

In [0]:
x_test1911_bin=[]
y_test1911_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test1911):
  if x_test1911[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test1911_bin.append(1)
        x_test1911_bin.append(x_test1911[i])
        smoking_event+=1
      elif y >= 20:
        y_test1911_bin.append(0)
        x_test1911_bin.append(x_test1911[i])
        non_smoking_event+=1
        
x_test1911_bin=np.array(x_test1911_bin)
y_test1911_bin=np.array(y_test1911_bin)

In [11]:
print(x_bin.shape)
print(y_bin.shape)
print(x_test1911_bin.shape)
print(y_test1911_bin.shape)

(83369, 160, 4)
(83369,)
(10908, 160, 4)
(10908,)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [13]:
!pip install keras-tcn

  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5e357d6728eb9ed8ebeb88a629327dcaf0dc41cb957e9ba9780a82068f4f36ec
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [14]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
from sklearn.utils import class_weight

from tcn import TCN, tcn_full_summary

batch_size, timesteps, input_dim = None, 160, 4


def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, timesteps, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0
    y_train[pos_indices, 0] = 1.0
    return x_train, y_train


i = Input(batch_shape=(batch_size, timesteps, input_dim))



o = TCN(return_sequences=True)(i)  # The TCN layers are here.
#o = TCN(return_sequences=False)(o)
#o = Conv1D(filters=128,activation = 'relu', kernel_size=3)(o)
#o = LayerNormalization()(o)
#o = Dropout(0.1)(o)
#o = Dense(64)(o)
#o = Dense(32)(o)
o = Dense(64, activation='relu')(o)
o = Dense(1, activation='sigmoid')(o)

#variable = {1:30, 0:1}

m = Model(inputs=[i], outputs=[o])
#m.compile(optimizer='adam', loss='mse')
m.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

tcn_full_summary(m, expand_residual_blocks=False)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 4)]          0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 160, 64), (None,  9152      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 160, 64), (None,  16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 160, 64), (None,  16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 160, 64), (None,  16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 160, 64), (None,  16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 160, 64), (None,  16512 

In [16]:
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_bin.shape[1],x_bin.shape[2])))
model.add(TCN(nb_filters=64,
              kernel_size=3,
              dilations=[1, 2, 4, 8, 16, 32, 64], 
              activation='relu',
              kernel_initializer='he_normal',))
model.add(Dropout(0.1))
#model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 158, 64)           832       
_________________________________________________________________
tcn_2 (TCN)                  (None, 64)                172928    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 175,873
Trainable params: 175,873
Non-trainable params: 0
________________________________________________

In [25]:
print("%s: %.2f%%" % (model.metrics_names[1], scores_model_seq[1]*100))

accuracy: 95.88%


In [17]:
history = model.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=10,verbose=0)
scores_model_seq = model.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (model.metrics_names[1], scores_model_seq[1]*100))

341/341 [==============================] - 15s 45ms/step - loss: 0.1294 - accuracy: 0.9588
accuracy: 95.88%


In [18]:
prediction_y  = model.predict_classes(x_test1911_bin)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [19]:
from sklearn.metrics import classification_report, f1_score
print(classification_report(y_test1911_bin,prediction_y))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     10188
           1       0.77      0.54      0.64       720

    accuracy                           0.96     10908
   macro avg       0.87      0.77      0.81     10908
weighted avg       0.95      0.96      0.96     10908



In [0]:
model.save("Model_TCN_freshAir.h5")

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
downloaded6 = drive.CreateFile({'id':"1QUowiiO_1UoFMv6r1tENHJwU218fwJjt"})   # replace the id with id of file you want to access
downloaded6.GetContentFile('x_test.p') 

#https://drive.google.com/open?id=1QUowiiO_1UoFMv6r1tENHJwU218fwJjt

In [0]:
downloaded7 = drive.CreateFile({'id':"1vC76GTPUy9HHzVPYKVwL7IvemWecesC9"})   # replace the id with id of file you want to access
downloaded7.GetContentFile('y_test.p') 

#https://drive.google.com/open?id=1vC76GTPUy9HHzVPYKVwL7IvemWecesC9

In [0]:
x_test1910 = pkl.load(open("x_test.p", "rb"))
y_test1910 = pkl.load(open("y_test.p", "rb"))

In [0]:
x_test1910_bin=[]
y_test1910_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test1910):
  if x_test1910[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test1910_bin.append(1)
        x_test1910_bin.append(x_test1910[i])
        smoking_event+=1
      elif y >= 20:
        y_test1910_bin.append(0)
        x_test1910_bin.append(x_test1910[i])
        non_smoking_event+=1
        
x_test1910_bin=np.array(x_test1910_bin)
y_test1910_bin=np.array(y_test1910_bin)

In [0]:

#save model in json format

from tensorflow.keras.models import Sequential, model_from_json

TCN_model_as_json = model.to_json()
with open(r'model.json', "w") as json_file:
    json_file.write(TCN_model_as_json)
# save weights to file (for this format, need h5py installed)
model.save_weights('weights.h5')

In [0]:
# relaoding the saved model
import json

loaded_json = open(r'model.json', 'r').read()
reloaded_model = model_from_json(loaded_json, custom_objects={'TCN': TCN})
reloaded_model.load_weights(r'weights.h5')

In [44]:
tcn_full_summary(reloaded_model, expand_residual_blocks=False)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 158, 64)           832       
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 158, 64), (None,  24704     
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 158, 64), (None,  24704     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 158, 64), (None,  24704     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 158, 64), (None,  24704     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 158, 64), (None,  24704     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 158, 64), (None, 

In [0]:
reload_pred_y = reloaded_model.predict_classes(x_test1910_bin)

In [52]:
#relaoded model reults
print(classification_report(y_test1910_bin,reload_pred_y))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10591
           1       0.81      0.55      0.66      1064

    accuracy                           0.95     11655
   macro avg       0.88      0.77      0.81     11655
weighted avg       0.94      0.95      0.94     11655



In [50]:
reload_pred_y.shape

(11655, 1)

In [0]:
prediction_y_1910  = model.predict_classes(x_test1910_bin)

In [51]:
y_test1910_bin.shape

(11655,)

In [36]:
print(classification_report(y_test1910_bin,prediction_y_1910))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10591
           1       0.81      0.55      0.66      1064

    accuracy                           0.95     11655
   macro avg       0.88      0.77      0.81     11655
weighted avg       0.94      0.95      0.94     11655



In [40]:
model2 = Sequential()
model2.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(x_bin.shape[1],x_bin.shape[2])))

model2.add(TCN(nb_filters=128,
              kernel_size=3,
              dilations=[1, 2, 4, 8, 16, 32, 64], 
              activation='relu',
              kernel_initializer='he_normal',))
model2.add(Dropout(0.1))
#model2.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 158, 128)          1664      
_________________________________________________________________
tcn_9 (TCN)                  (None, 128)               689920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [41]:
history = model2.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=10,verbose=0)
scores_model_seq2 = model2.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (model2.metrics_names[1], scores_model_seq2[1]*100))

341/341 [==============================] - 36s 107ms/step - loss: 0.1223 - accuracy: 0.9573
accuracy: 95.73%


In [18]:
pred_y

array([[[5.59951663e-02],
        [3.46069336e-02],
        [4.55340743e-02],
        ...,
        [9.40599322e-01],
        [9.39576507e-01],
        [9.39603686e-01]],

       [[9.22623277e-02],
        [1.13067538e-01],
        [1.17221296e-01],
        ...,
        [8.90118718e-01],
        [8.72622192e-01],
        [8.18548203e-01]],

       [[2.52202570e-01],
        [2.12323606e-01],
        [3.51065606e-01],
        ...,
        [4.29025143e-01],
        [3.23843837e-01],
        [2.52877235e-01]],

       ...,

       [[2.76222825e-03],
        [1.17334723e-03],
        [2.60937214e-03],
        ...,
        [1.23402476e-03],
        [2.11334229e-03],
        [1.24052167e-03]],

       [[6.37233257e-04],
        [1.10438466e-03],
        [1.33287907e-03],
        ...,
        [5.65600224e-08],
        [2.81003452e-07],
        [2.59424127e-07]],

       [[2.16230750e-03],
        [8.08775425e-04],
        [3.58223915e-04],
        ...,
        [4.27717277e-05],
        [2.5626

In [15]:
.# 1 TCN layer
history = m.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=10,verbose=0)
scores_model3 = m.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (m.metrics_names[1], scores_model3[1]*100))

341/341 [==============================] - 11s 34ms/step - loss: 1.0178 - accuracy: 0.9340
accuracy: 93.40%


In [0]:
# 2 TCN layer

history = m.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=10,verbose=0)
scores_model3 = m.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (m.metrics_names[1], scores_model3[1]*100))

341/341 [==============================] - 25s 74ms/step - loss: 1.0178 - accuracy: 0.9340
accuracy: 93.40%


In [0]:
history = m.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=25,verbose=0)
scores_model3 = m.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (m.metrics_names[1], scores_model3[1]*100))

341/341 [==============================] - 13s 39ms/step - loss: 1.0178 - accuracy: 0.9340
accuracy: 93.40%
